In [23]:
import pandas as pd
# Reload the datasets with the correct delimiter
math_data = pd.read_csv('data/student-mat.csv', delimiter=';')
portuguese_data = pd.read_csv('data/student-por.csv', delimiter=';')

# Verify the structure by displaying the first few rows of each dataset again
math_data.head(), portuguese_data.head()


(  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
 0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
 1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
 2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
 3     GP   F   15       U     GT3       T     4     2   health  services  ...   
 4     GP   F   16       U     GT3       T     3     3    other     other  ...   
 
   famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
 0      4        3      4     1     1      3        6   5   6   6  
 1      5        3      3     1     1      3        4   5   5   6  
 2      4        3      2     2     3      3       10   7   8  10  
 3      3        2      2     1     1      5        2  15  14  15  
 4      4        3      2     1     2      5        4   6  10  10  
 
 [5 rows x 33 columns],
   school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjo

In [17]:
# Define the columns to create a unique student ID
id_cols = ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 
           'Mjob', 'Fjob', 'reason', 'nursery', 'internet']

# Create 'student_id' for both datasets
math_data['student_id'] = math_data[id_cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
portuguese_data['student_id'] = portuguese_data[id_cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

# Append 'student_id' in id_col
id_cols.append('student_id')

# Merge the datasets on 'student_id'

merged_data = pd.merge(math_data, portuguese_data, on=id_cols, suffixes=('_mat', '_por'), how='outer')

# Check the structure and first few rows of the merged dataset
merged_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 54 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   school          682 non-null    object 
 1   sex             682 non-null    object 
 2   age             682 non-null    int64  
 3   address         682 non-null    object 
 4   famsize         682 non-null    object 
 5   Pstatus         682 non-null    object 
 6   Medu            682 non-null    int64  
 7   Fedu            682 non-null    int64  
 8   Mjob            682 non-null    object 
 9   Fjob            682 non-null    object 
 10  reason          682 non-null    object 
 11  guardian_mat    407 non-null    object 
 12  traveltime_mat  407 non-null    float64
 13  studytime_mat   407 non-null    float64
 14  failures_mat    407 non-null    float64
 15  schoolsup_mat   407 non-null    object 
 16  famsup_mat      407 non-null    object 
 17  paid_mat        407 non-null    obj

In [13]:
combined_df = merged_data

# Correctly append '_0' to all student_ids in the combined_df to represent original entries
combined_df['student_id'] = combined_df['student_id'].apply(lambda x: x + '_0')

# Check the first few rows to confirm the change
combined_df[['student_id']].head()


,student_id
0,GP_F_18_U_GT3_A_4_4_at_home_teacher_course_yes...
1,GP_F_17_U_GT3_T_1_1_at_home_other_course_no_yes_0
2,GP_F_15_U_LE3_T_1_1_at_home_other_other_yes_yes_0
3,GP_F_15_U_GT3_T_4_2_health_services_home_yes_y...
4,GP_F_16_U_GT3_T_3_3_other_other_home_yes_no_0


In [14]:
# Identify and print the duplicate student_ids
duplicate_student_ids = combined_df[combined_df.duplicated(subset='student_id', keep=False)]

# Group the duplicate entries by student_id and compare the rows within each group
duplicate_groups = duplicate_student_ids.groupby('student_id')

# Inspect differences within each group
differences = {}
for student_id, group in duplicate_groups:
    if not group.equals(group.iloc[0]):
        differences[student_id] = group

# Show differences for the first few student_ids
for student_id, group in list(differences.items())[:]: 
    print(f"Differences for student_id: {student_id}")
    print(group)
    print("\n")

Differences for student_id: GP_F_17_U_GT3_T_4_4_teacher_services_course_yes_yes_0
    school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
339     GP   F   17       U     GT3       T     4     4  teacher  services   
340     GP   F   17       U     GT3       T     4     4  teacher  services   

     ... famrel_por freetime_por  goout_por  Dalc_por  Walc_por health_por  \
339  ...        5.0          3.0        1.0       1.0       4.0        5.0   
340  ...        5.0          4.0        4.0       1.0       3.0        4.0   

    absences_por G1_por G2_por G3_por  
339          2.0   11.0   11.0   12.0  
340          0.0   13.0   12.0   13.0  

[2 rows x 54 columns]


Differences for student_id: GP_F_18_U_GT3_T_1_1_other_other_home_yes_yes_0
    school sex  age address famsize Pstatus  Medu  Fedu   Mjob   Fjob  ...  \
293     GP   F   18       U     GT3       T     1     1  other  other  ...   
294     GP   F   18       U     GT3       T     1     1  other  other  .

In [15]:
# Append '_0' to all student_ids in both the math and portuguese datasets to indicate original entries
math_data['student_id'] = math_data['student_id'].apply(lambda x: x + '_0')
portuguese_data['student_id'] = portuguese_data['student_id'].apply(lambda x: x + '_0')

# Verify the changes by displaying the first few rows of both datasets
math_data[['student_id']].head(), portuguese_data[['student_id']].head()


(                                          student_id
 0  GP_F_18_U_GT3_A_4_4_at_home_teacher_course_yes...
 1  GP_F_17_U_GT3_T_1_1_at_home_other_course_no_yes_0
 2  GP_F_15_U_LE3_T_1_1_at_home_other_other_yes_yes_0
 3  GP_F_15_U_GT3_T_4_2_health_services_home_yes_y...
 4      GP_F_16_U_GT3_T_3_3_other_other_home_yes_no_0,
                                           student_id
 0  GP_F_18_U_GT3_A_4_4_at_home_teacher_course_yes...
 1  GP_F_17_U_GT3_T_1_1_at_home_other_course_no_yes_0
 2  GP_F_15_U_LE3_T_1_1_at_home_other_other_yes_yes_0
 3  GP_F_15_U_GT3_T_4_2_health_services_home_yes_y...
 4      GP_F_16_U_GT3_T_3_3_other_other_home_yes_no_0)

Now for each student_id duplicate row check these columns'
'paid', 'famrel', 'freetime', 'failures', 'goout', 'studytime', 'health', 'Dalc', 'higher', 'guardian', 'Walc', 'romantic', 'famsup', 'schoolsup', 'traveltime', 'activities' of the _mat and _por versions within that row
also keep a counter starting from 0 for each student_id
if they match then keep them and manipulate the last  them the counter at the end of the student . If they don't keep them both. As each group is resolved Also simultaneously save the student_ids that have been manipulated o

1. for each student_id in the differences find them in math_data and portuguese_data. I'm sure you'll find multiple rows representing same student_id in both.
2. save these rows in seperate groups saying _mat and _por from the math_data and portuguese_data. and delete all the duplicates from the combined_df
3. compare these columns 'paid', 'famrel', 'freetime', 'failures', 'goout', 'studytime', 'health', 'Dalc', 'higher', 'guardian', 'Walc', 'romantic', 'famsup', 'schoolsup', 'traveltime', 'activities' of both the groups
4. wherever they match therefore they are the same student so manipulate the last integer in the student_id by changing the integer value(so that the same students get matching id in both groups also change the student_id in the math_data and portuguese_data for the respective student) so add them to the combined_df acc. to its format of columns by copying the columns _mat and _por from the respective student_id to the combined_df and update the respective student_id.
5. do this for all the duplicates and this should resolve the issue of duplicates

In [18]:
# Step 1: Identify and separate entries from math_data and portuguese_data based on differences found
student_ids_with_diffs = list(differences.keys())

# Find entries in math_data and portuguese_data for each student_id with differences
math_groups = {}
portuguese_groups = {}

for student_id in student_ids_with_diffs:
    original_id = student_id[:-2]  # Remove the '_0' to match against the original data
    math_groups[student_id] = math_data[math_data['student_id'] == original_id]
    portuguese_groups[student_id] = portuguese_data[portuguese_data['student_id'] == original_id]

# Example print to verify the separation into groups (showing one example)
math_groups[student_ids_with_diffs[0]], portuguese_groups[student_ids_with_diffs[0]]

(    school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
 328     GP   F   17       U     GT3       T     4     4  teacher  services   
 
      ... freetime goout  Dalc  Walc  health absences  G1 G2 G3  \
 328  ...        4     4     1     3       4        7  10  9  9   
 
                                             student_id  
 328  GP_F_17_U_GT3_T_4_4_teacher_services_course_ye...  
 
 [1 rows x 34 columns],
     school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
 271     GP   F   17       U     GT3       T     4     4  teacher  services   
 380     GP   F   17       U     GT3       T     4     4  teacher  services   
 
      ... freetime goout  Dalc  Walc  health absences  G1  G2  G3  \
 271  ...        3     1     1     4       5        2  11  11  12   
 380  ...        4     4     1     3       4        0  13  12  13   
 
                                             student_id  
 271  GP_F_17_U_GT3_T_4_4_teacher_services_course_ye... 

In [19]:
# Step 2: Remove the duplicate entries from combined_df
for student_id in student_ids_with_diffs:
    # Remove entries matching the duplicate student_id
    combined_df = combined_df[combined_df['student_id'] != student_id]

# Verify by checking if any of the duplicate student_ids still exist in combined_df
remaining_duplicates_check = combined_df[combined_df['student_id'].isin(student_ids_with_diffs)]
remaining_duplicates_check


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel_por,freetime_por,goout_por,Dalc_por,Walc_por,health_por,absences_por,G1_por,G2_por,G3_por


In [21]:
# Specified columns for comparison
comparison_columns = ['paid', 'famrel', 'freetime', 'failures', 'goout', 'studytime', 
                      'health', 'Dalc', 'higher', 'guardian', 'Walc', 'romantic', 
                      'famsup', 'schoolsup', 'traveltime', 'activities']

# Function to check if entries across datasets match on specified columns
def entries_match(row1, row2):
    return all(row1[col] == row2[col] for col in comparison_columns)

# Adjusting the function to manually create a new row from both math and Portuguese data entries

def handle_duplicates_and_combine_adjusted(math_group, port_group, combined_df):
    for _, math_row in math_group.iterrows():
        for _, port_row in port_group.iterrows():
            if entries_match(math_row, port_row):
                # Create a unified student_id
                unified_id = math_row['student_id'][:-2] + "_1"

                # Update the original dataframes
                math_data.loc[math_data['student_id'] == math_row['student_id'], 'student_id'] = unified_id
                portuguese_data.loc[portuguese_data['student_id'] == port_row['student_id'], 'student_id'] = unified_id

                # Manually create a new row combining both math and Portuguese data
                new_row_data = {**math_row.to_dict(), **port_row.to_dict(), 'student_id': unified_id}
                new_combined_row = pd.DataFrame([new_row_data], columns=combined_df.columns)

                # Append the new row to the combined_df
                combined_df = pd.concat([combined_df, new_combined_row], ignore_index=True)

                break  # Stop checking after the first match
    return combined_df

# Apply the adjusted function to each pair of groups
for student_id, math_group in math_groups.items():
    port_group = portuguese_groups[student_id]
    combined_df = handle_duplicates_and_combine_adjusted(math_group, port_group, combined_df)

# Display the first few updated entries in combined_df to verify changes
combined_df.head()


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel_por,freetime_por,goout_por,Dalc_por,Walc_por,health_por,absences_por,G1_por,G2_por,G3_por
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4.0,3.0,4.0,1.0,1.0,3.0,4.0,0.0,11.0,11.0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5.0,3.0,3.0,1.0,1.0,3.0,2.0,9.0,11.0,11.0
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4.0,3.0,2.0,2.0,3.0,3.0,6.0,12.0,13.0,12.0
3,GP,F,15,U,GT3,T,4,2,health,services,...,3.0,2.0,2.0,1.0,1.0,5.0,0.0,14.0,14.0,14.0
4,GP,F,16,U,GT3,T,3,3,other,other,...,4.0,3.0,2.0,1.0,2.0,5.0,0.0,11.0,13.0,13.0


In [22]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656 entries, 0 to 655
Data columns (total 54 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   school          656 non-null    object 
 1   sex             656 non-null    object 
 2   age             656 non-null    int64  
 3   address         656 non-null    object 
 4   famsize         656 non-null    object 
 5   Pstatus         656 non-null    object 
 6   Medu            656 non-null    int64  
 7   Fedu            656 non-null    int64  
 8   Mjob            656 non-null    object 
 9   Fjob            656 non-null    object 
 10  reason          656 non-null    object 
 11  guardian_mat    383 non-null    object 
 12  traveltime_mat  383 non-null    float64
 13  studytime_mat   383 non-null    float64
 14  failures_mat    383 non-null    float64
 15  schoolsup_mat   383 non-null    object 
 16  famsup_mat      383 non-null    object 
 17  paid_mat        383 non-null    obj